# 태스크 1b: 컨텍스트가 포함된 프롬프트를 사용하여 텍스트 생성 수행

이 노트북에서는 고객 지원 엔지니어로부터 받은 고객 서비스의 품질에 만족하지 못한 고객에게 보내는 이메일 응답을 생성하는 방법을 알아보게 됩니다. 불만족스러운 고객으로부터 받은 실제 이메일의 콘텐츠를 포함하여 모델에 추가 컨텍스트를 제공하게 됩니다.

PromptTemplate의 도움을 받아 복잡성을 더 추가하여 비슷한 사용 사례에 LangChain 프레임워크를 활용하게 됩니다. PromptTemplate을 사용하면 나중에 정보로 채울 수 있는 일반 셸을 생성하고 다양한 시나리오에 따라 모델 출력을 얻을 수 있습니다.

[LangChain](https://python.langchain.com/docs/tutorials/)은 언어 모델이 지원하는 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크의 핵심적인 측면은 다양한 구성 요소를 함께 연결하여 고급 사용 사례를 생성함으로써 대규모 언어 모델을 증강할 수 있다는 것입니다.

프롬프트의 추가 컨텍스트로 인해 이 노트북에서 생성된 텍스트는 이전에 제로샷 프롬프트를 통해 생성된 콘텐츠보다 품질과 관련성이 훨씬 더 좋습니다. 이 노트북에서 사용된 프롬프트에서는 텍스트 생성 요청에 컨텍스트를 추가할 사용자 지정 LangChain 프롬프트 템플릿을 생성합니다.

#### 시나리오
당신은 AnyCompany의 고객 서비스 관리자인 Bob이며, 일부 고객이 고객 서비스에 만족하지 못하고 고객 지원 엔지니어가 제공하는 서비스에 대한 부정적인 피드백을 제공하고 있습니다. 이제 해당 고객에게 겸손하게 대응하여 서비스 불만족에 대해 사과하고 신뢰를 다시 되찾고 싶습니다. 이전 이메일 기반으로 고객의 정서에 맞게 인간 친화적이고 개인화된 대량의 이메일을 생성하려면 대규모 언어 모델(LLM)의 도움이 필요합니다.

이 시나리오에서는 LangChain의 PromptTemplate 기능을 활용하여 고객의 이전 이메일을 기반으로 개인화된 이메일 응답을 생성하기 위한 일반 셸을 만들 수 있습니다. PromptTemplate은 고객의 원본 이메일 콘텐츠를 통합하여 LLM이 컨텍스트와 감정을 이해한 다음에 관련성 있는 맞춤형 응답을 생성할 수 있도록 합니다.

## 태스크 1b.1: 환경 설정

이 태스크에서는 환경을 설정합니다.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))


bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## 태스크 1b.2: Bedrock LLM 모델 호출

이 태스크에서는 LLM으로부터 Bedrock 클래스의 인스턴스를 생성합니다. 여기에는 Amazon Bedrock에서 사용할 수 있는 모델의 식별자인 `model_id`가 필요합니다.

선택 사항으로, 이전에 생성한 boto3 클라이언트와 `temperature`, `top_p`, `max_token_count` 또는 `stop_sequences`와 같은 파라미터를 담을 수 있는 `model_kwargs`도 전달할 수 있습니다(파라미터에 대한 자세한 내용은 Amazon Bedrock 콘솔에서 확인할 수 있습니다).

Amazon Bedrock에서 사용 가능한 텍스트 생성 모델 ID는 [설명서](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html)를 참조하십시오.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 모델마다 다른 `model_kwargs`를 지원합니다.

In [ ]:
# Model configuration
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

model_id = "meta.llama3-8b-instruct-v1:0"
model_kwargs =  { 
        "max_gen_len": 512,
        "temperature": 0,
        "top_p": 1,
}

# LangChain class for chat
chat_model = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

## 태스크 1b.3: LangChain 사용자 지정 프롬프트 템플릿 생성

이 태스크에서는 실행할 때마다 다른 입력 변수를 전달할 수 있는 프롬프트용 템플릿을 생성합니다. 이는 데이터베이스에서 가져올 수 있는 다양한 입력 변수로 콘텐츠를 생성해야 하는 경우에 유용합니다.

이전 태스크에서는 프롬프트를 하드코딩했습니다. 여러 고객이 비슷한 부정적 피드백을 보내는 경우, 이제 각 고객의 이메일을 사용하여 사과하는 응답을 보내되 응답을 개인화하고자 하는 경우가 있을 수 있습니다. 다음 셀에서는 `PromptTemplate`을 생성하여 이 패턴을 달성할 수 있는 방법을 살펴보게 됩니다.

In [ ]:
# Create a prompt template that has multiple input variables
from langchain.prompts import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} at AnyCompany to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob Smith", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
# get number of tokens
num_tokens = chat_model.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 경고를 무시하고 다음 셀로 넘어가도 됩니다.

In [ ]:
#invoke
response = chat_model.invoke(prompt)

In [ ]:
# Configure a Chain to parse output
chain = StrOutputParser()
formatted_response=chain.invoke(response)
print(formatted_response)

컨텍스트 없이 LLM을 호출하면 원하는 결과를 얻지 못할 수 있다는 것을 알았습니다. 컨텍스트를 추가하고 프롬프트 템플릿을 더 사용하여 LLM의 출력을 제한하면 원하는 결과를 얻을 수 있었습니다.

### 직접 해보기
- 프롬프트를 특정 사용 사례로 변경하고 다양한 모델의 출력을 평가하십시오.
- 토큰 길이를 조작하면서 서비스의 지연 시간 및 응답성을 이해해 보십시오.
- 다양한 프롬프트 엔지니어링 원칙을 적용하여 더 나은 출력을 얻을 수 있습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가서 **태스크 2**를 계속 진행합니다.